# Деревья решений

## Построение дерева

Опишем жадный алгоритм построения бинарного дерева решений:
1. Начинаем со всей обучающей выборки $X$, которую помещаем в корень $R_1$. 
2. Задаём функционал качества $Q(X, j, t)$ и критерий остановки. 
3. Запускаем построение из корня: $SplitNode(1, R_1)$

Функция $SplitNode(m, R_m)$
1. Если выполнен критерий остановки, то выход.
2. Находим наилучший с точки зрения $Q$ предикат: $j, t$: $[x_j<t]$
3. Помещаем предикат в вкршину и получаем с его помощью разбиение $X$ на две части: $R_{left} = \lbrace x|x_j<t \rbrace$ и $R_{right} = \lbrace x|x_j \geqslant t \rbrace$
4. Поместим $R_{left}$ и $R_{right}$ соответсвенно в левое и правое поддерево.
5. Рекурсивно повторяем $SplitNode(left, R_{left})$ и $SplitNode(right, R_{right})$.

В конце поставим в соответствие каждому листу ответ. Для задачи классификации - это самый частый среди объектов класс или вектор с долями классов (можно интерпретировать как вероятности):
$$ c_v = \arg \max_{k\in Y} \sum_{(x_i,y_i) \in R_v} [y_i=k]  $$

## Функционал качества для деревьев решений


Энтропия Шеннона для системы с N возможными состояниями определяется по формуле:
$$H = - \sum_{i=0}^{N} p_i\log_2p_i $$

где $p_i$ – вероятности нахождения системы в $i$-ом состоянии. 

Это очень важное понятие теории информации, которое позволяет оценить количество информации (степень хаоса в системе). Чем выше энтропия, тем менее упорядочена система и наоборот. С помощью энтропии мы формализуем функционал качества для разделение выборки (для задачи классификации).

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import random
from pprint import pprint

Код для расчёта энтропии:

In [ ]:
def entropy(y):
    
    _, counts = np.unique(y, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
     
    return entropy

Здесь $y$ - это массив значений целевой переменной

Энтропия – по сути степень хаоса (или неопределенности) в системе. Уменьшение энтропии называют приростом информации (information gain, IG).

Обочначим $R_v$ - объекты, которые нужно разделить в помощью предиката в вершине $v$. Запишем формулу для расчёта информационного прироста:
$$ Q = IG = H(R_v) - (H(R_{left})+H(R_{right}))$$

На каждом шаге нам нужно максимизировать этот функционал качества. Как это делать? Например, так можно перебрать $t$ для выбранного $j$.

Предыдущая версия формулы прироста информации слишком упрощена. В работе необходимо использовать более устойчивую формулу, которая учитывает не только энтропию подмножеств, но и их размер. 

$$ Q = IG = H(R_v) - \Big (\frac{|R_{left}|} {|R_{v}|} H(R_{left})+ \frac{|R_{right}|} {|R_{v}|} H(R_{right})\Big)$$

где, $|R_{v}|$, $|R_{left}|$ и $|R_{right}|$ - количество элементов в соответствующих множествах.


### Задание 4.1

Реализуйте алгоритм построения дерева. Должны быть отдельные функции (методы) для расчёта энтропии (уже есть), для разделения узлов дерева (используйте, например, `pandas`), для подсчёта функционала качества $IG$, для выбора наилучшего разделения (с учетом признаков и порогов), для проверки критерия остановки.

Для набора данных `iris` реализуйте алгоритм и минимум три из разными критерия остановки из перечисленных ниже:
* максимальной глубины дерева = 5
* минимального числа объектов в листе = 5
* максимальное количество листьев в дереве = 5
* purity (остановка, если все объекты в листе относятся к одному классу)

Реализуйте функцию `predict` (на вход функции подаётся датафрейм с объектами)

Оцените точность каждой модели с помощью метрики доля правильных ответов (`from sklearn.metrics import accuracy_score` или реализовать свою).

In [ ]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from collections import Counter

In [ ]:
def inf_gain(v, left, right):
    r_v = len(v)
    r_left = len(left)
    r_right = len(right)
    ig = entropy(v) - (r_left / r_v * entropy(left) + r_right / r_v * entropy(right))
    return ig

In [ ]:
def split(column, y):
    best_ig = -1
    best_t = None

    for value in set(column):
        y_predict = column < value
        ig = inf_gain(y, y[y_predict], y[~y_predict])
        if ig > best_ig:
            best_t, best_ig = value, ig
    return best_t, best_ig

In [ ]:
def best_split(X, y):
    best_j, best_t = None, None
    best_ig = -1

    for i, c in enumerate(X.T):
        t, ig = split(c, y)
        if ig > best_ig:
            best_t, best_ig = t, ig
            best_j = i
    return best_t, best_ig, best_j

In [ ]:
class Node:
    def __init__(self, number=1, depth=1):
        self.number = number
        self.depth = depth
        self.is_leaf = False
        self.left = None
        self.right = None
        self.ig = 0
        self.t = 0
        self.j = 0
        self.main_class = None

    def return_class(self):
        return self.main_class

In [ ]:
class DecisionTree:
    def __init__(self):
        self.min_leaf_size = 1
        self.max_depth = 1000
        self.max_leaf_count = 10000
        self.purity = False
        self.leaf_count = 0
        self.tree_root = None
        self.real_depth = 0

    def _set_criteria(self, criteria):
        if criteria == "max_depth":
            self.max_depth = 5
        elif criteria == "purity":
            self.purity = True
        elif criteria == "min_leaf_size":
            self.min_leaf_size = 5
        elif criteria == "max_leaf_count":
            self.max_leaf_count = 5
        else:
            raise Exception("Incorrect criteria! Model doesn't fit")

    def _stop(self, node, y):
        if self.purity is True and len(set(y)) == 1:
            return True
        elif node.depth == self.max_depth:
            return True
        elif len(y) <= self.min_leaf_size:
            return True
        elif self.leaf_count >= self.max_leaf_count:
            return True
        else:
            return False

    def _split_node(self, node, x, y):
        if self._stop(node, y):
            if len(y) == 0:
                self.leaf_count -= 1
                return None
            if node.depth > self.real_depth:
                self.real_depth = node.depth
            node.is_leaf = True
            b = Counter(y)
            node.main_class = b.most_common(1)[0][0]
            return node
        node.t, node.ig, node.j = best_split(x, y)
        x1, y1 = x[x[:, node.j] < node.t], y[x[:, node.j] < node.t]
        x2, y2 = x[x[:, node.j] >= node.t], y[x[:, node.j] >= node.t]
        if len(y1) == 0 or len(y2) == 0:
            if node.depth > self.real_depth:
                self.real_depth = node.depth
            node.is_leaf = True
            b = Counter(y)
            node.main_class = b.most_common(1)[0][0]
            return node
        node_left = Node(node.number*2, node.depth+1)
        node_right = Node(node.number * 2 + 1, node.depth + 1)
        self.leaf_count += 1
        node.left = self._split_node(node_left, x1, y1)
        node.right = self._split_node(node_right, x2, y2)
        return node

    def fit(self, x, y, criteria):
        try:
            self._set_criteria(criteria)
        except Exception as e:
            print(e)
            return False
        root = Node()
        self.leaf_count = 1
        self.tree_root = self._split_node(root, x, y)
        if self.tree_root is None:
            return False
        else:
            return True

    def _get_prediction(self, row):
        rt = self.tree_root
        while rt.is_leaf is False:
            if row[rt.j] < rt.t:
                rt = rt.left
            elif row[rt.j] >= rt.t:
                rt = rt.right
        return rt.main_class

    def predict(self, x):
        results = np.array([0] * len(x))
        for i, c in enumerate(x):
            results[i] = self._get_prediction(c)
        return results

In [ ]:
iris = datasets.load_iris()

X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.33, random_state=42)

dt = DecisionTree()
res = dt.fit(X_train, y_train, 'max_depth')
y_pred = dt.predict(X_test)

dt2 = DecisionTree()
res2 = dt2.fit(X_train, y_train, 'purity')
y2_pred = dt2.predict(X_test)

dt3 = DecisionTree()
res3 = dt3.fit(X_train, y_train, 'min_leaf_size')
y3_pred = dt3.predict(X_test)

dt4 = DecisionTree()
res4 = dt4.fit(X_train, y_train, 'max_leaf_count')
y4_pred = dt4.predict(X_test)


print("Accuracy score: max_depth ", accuracy_score(y_test, y_pred))
print("Accuracy score: purity ", accuracy_score(y_test, y2_pred))
print("Accuracy score: min_leaf_size ", accuracy_score(y_test, y3_pred))
print("Accuracy score: max_leaf_count ", accuracy_score(y_test, y4_pred))

Accuracy score: max_depth  0.98
Accuracy score: purity  0.96
Accuracy score: min_leaf_size  0.96
Accuracy score: max_leaf_count  0.98


##  Случайный лес

Опишем алгоритм случайный лес (*random forest*) и попутно разберём основные идеи:

1. Зададим $N$ - число деревьев в лесу.
2. Для каждого $n$ из $N$ сгенерируем свою выборку $X_n$. Пусть $m$ - это количество объектов в $X$. При генерации каждой $X_n$ мы будем брать объекты $m$ раз с возвращением. То есть один и тот же объект может попасть в выборку несколько раз, а какие-то объекты не попадут. (Этот способ назвается бутстрап).
3. По каждой $X_n$ построим решающее дерево $b_n$. Обычно стараются делать глубокие деревья. В качестве критериев остановки можно использовать `max_depth` или `min_samples_leaf` (например, пока в каждом листе не окажется по одному объекту). При каждом разбиении сначала выбирается $k$ (эвристика $k = \sqrt d$, где $d$ - это число признаков объектов из выборки $X$) случайных признаков из исходных, и оптимальное разделение выборки ищется только среди них. Обратите внимание, что мы не выбрасываем оставшиеся признаки!
4. Итоговый алгоритм будет представлять собой результат голосования (для классификации) и среднее арифметическое (для регрессии). Модификация алгоритма предполагает учёт весов каждого отдельного слабого алгоритма в ансамбле, но в этом особо нет смысла.


### Задание 4.2

В качестве набора данных используйте: https://www.kaggle.com/mathchi/churn-for-bank-customers

Там есть описание и примеры работы с этими данными. Если кратко, речь идёт про задачу прогнозирования оттока клиентов. Есть данные о 10 тысячах клиентов банка, часть из которых больше не являются клиентами.

Используя либо свою реализацию, либо  `DecisionTreeClassifier` с разными настройками из `sklearn.tree` реализйте алгоритм "случайный лес". 

Найдите наилучшие гиперпараметры этого алгоритма: количество деревьев, критерий остановки, функционал качества, минимальное количество объектов в листьях и другие.

Нельзя использовать готовую реализацию случайного леса из `sklearn`.

В подобных задачах очень важна интерпретируемость алгоритма. Попытайтесь оценить информативность признаков, т.е. ответить а вопрос, значения каких признаков являются самыми важными индикаторами того, что банк потеряет клиента.

In [ ]:
import pandas as pd 
df = pd.read_csv("/content/sample_data/churn.csv")

In [ ]:
df.sample(5)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
5658,747,0.166734,1,41,9,0.00,1,1,0,32430.94,1
3246,638,0.161548,1,57,6,0.00,1,1,0,33676.48,1
4674,850,0.324432,1,41,1,176958.46,2,0,1,125806.30,0
9079,516,0.166734,0,27,1,0.00,1,0,1,112311.15,0
9579,821,0.324432,1,45,0,135827.33,2,1,1,131778.58,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [ ]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
RowNumber,10000.0,5.000500e+03,2886.895680,1.00,2500.75,5.000500e+03,7.500250e+03,10000.00
CustomerId,10000.0,1.569094e+07,71936.186123,15565701.00,15628528.25,1.569074e+07,1.575323e+07,15815690.00
CreditScore,10000.0,6.505288e+02,96.653299,350.00,584.00,6.520000e+02,7.180000e+02,850.00
Age,10000.0,3.892180e+01,10.487806,18.00,32.00,3.700000e+01,4.400000e+01,92.00
Tenure,10000.0,5.012800e+00,2.892174,0.00,3.00,5.000000e+00,7.000000e+00,10.00
Balance,10000.0,7.648589e+04,62397.405202,0.00,0.00,9.719854e+04,1.276442e+05,250898.09
NumOfProducts,10000.0,1.530200e+00,0.581654,1.00,1.00,1.000000e+00,2.000000e+00,4.00
HasCrCard,10000.0,7.055000e-01,0.455840,0.00,0.00,1.000000e+00,1.000000e+00,1.00
IsActiveMember,10000.0,5.151000e-01,0.499797,0.00,0.00,1.000000e+00,1.000000e+00,1.00
EstimatedSalary,10000.0,1.000902e+05,57510.492818,11.58,51002.11,1.001939e+05,1.493882e+05,199992.48


In [ ]:
df = df.drop(labels=['RowNumber', 'CustomerId', 'Surname'], axis=1)

df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [ ]:
df.Gender.replace(['Male', 'Female'], [1, 0], inplace=True)

df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [ ]:
!pip install category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 563 kB/s 


In [ ]:
from category_encoders import TargetEncoder
encoder = TargetEncoder()
df['Geography'] = encoder.fit_transform(df['Geography'], df['Exited'])

df

/usr/local/lib/python3.7/dist-packages/category_encoders/target_encoder.py:124: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/category_encoders/target_encoder.py:129: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  category=FutureWarning)


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,0.161548,0,42,2,0.00,1,1,1,101348.88,1
1,608,0.166734,0,41,1,83807.86,1,0,1,112542.58,0
2,502,0.161548,0,42,8,159660.80,3,1,0,113931.57,1
3,699,0.161548,0,39,1,0.00,2,0,0,93826.63,0
4,850,0.166734,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,0.161548,1,39,5,0.00,2,1,0,96270.64,0
9996,516,0.161548,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,0.161548,0,36,7,0.00,1,0,1,42085.58,1
9998,772,0.324432,1,42,3,75075.31,2,1,0,92888.52,1


In [ ]:
from sklearn.base import BaseEstimator
from sklearn.tree import DecisionTreeClassifier
from collections import Counter


class RandomForest(BaseEstimator):
    def __init__(self, n_trees=10, max_depth=50, min_samples_leaf = 1,
                 min_samples_split=2, criterion='entropy', max_leaf_nodes=25):
        self.trees = []
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.min_samples_split = min_samples_split
        self.criterion = criterion
        self.max_leaf_nodes = max_leaf_nodes

    def _bootstrap(self, X, y):
        n_samples = len(X)
        indexes = np.random.choice(n_samples, n_samples, replace=True)
        return X.iloc[indexes, :], y.iloc[indexes]


    def fit(self, X, y):
        self.trees = []
        for i in range(self.n_trees):
            tree = DecisionTreeClassifier(max_depth=self.max_depth,
                                          min_samples_leaf = self.min_samples_leaf,
                                          min_samples_split=self.min_samples_split,
                                          criterion=self.criterion,
                                          max_leaf_nodes=self.max_leaf_nodes,
                                          max_features='sqrt')
            _x, _y = self._bootstrap(X, y)
            tree.fit(_x, _y)
            self.trees.append(tree)

    def predict(self, X):
        predictions = np.swapaxes(np.array([tree.predict(X) for tree in self.trees]), 0, 1)
        final_predictions = [Counter(tree_pred).most_common(1)[0][0] for tree_pred in predictions]
        return np.array(final_predictions)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import numpy as np



X = df.iloc[:,:-1]
y = df.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
rd = RandomForest()

params = {
    'n_trees': np.arange(1, 50, 5),
    'max_depth': np.arange(5, 25, 3),
    'criterion' : ['gini', 'entropy']

}

params_prototype = {
    'n_trees': np.arange(1, 100, 5),
    'max_depth': np.arange(5, 24, 3),
    'min_samples_leaf' : np.arange(1, 4, 1),
    'min_samples_split' : np.arange(2, 10, 2),
    'criterion' : ['gini', 'entropy'],
    'max_leaf_nodes' : np.arange(25, 35, 2)
    
}

search = GridSearchCV(rd, params, scoring='accuracy')
search.fit(X_train, y_train)

print(search.best_params_)

{'max_depth': 14, 'n_trees': 36}


In [ ]:
best_tree = RandomForest(n_trees=36,max_depth=14)
best_tree.fit(X_train, y_train)
answers = best_tree.predict(X_test)

In [ ]:
from sklearn 

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
print("Accuracy score of tree with best params ", accuracy_score(y_test, answers))

Accuracy score of tree with best params  0.8645454545454545


In [ ]:
best_model_trees = search.best_estimator_.trees

feature = 0

for tree in best_model_trees:
  feature += tree.feature_importances_

feature /= search.best_estimator_.n_trees

importance = pd.DataFrame(data={
    'Attribute': X.columns,
    'Importance': feature
})

importance = importance.sort_values(by='Importance', ascending=False)

importance.head()

,Attribute,Importance
3,Age,0.440732
6,NumOfProducts,0.275179
8,IsActiveMember,0.090366
1,Geography,0.069935
5,Balance,0.067302
